### Library


In [1]:
import copy
import itertools as it
import networkx as nx
import numpy as np
import pandas as pd


### Input panel

In [2]:
userCode = "41264024354"


### Data convertion 

In [3]:
df = pd.read_csv("DPTSP.csv", header=None, dtype=int)
df = df.applymap(lambda x: int(userCode[x-1]) if x != 0 else x)
df.index = (df.index + 1).astype(str)
df.columns = (df.columns + 1).astype(str)

In [4]:
G = nx.from_pandas_adjacency(df, create_using=nx.DiGraph())
node = sorted(G.nodes(data=True))


### New Adjacency Matrix

In [5]:
print(df)


   1  2  3  4  5
1  0  2  4  4  6
2  4  0  4  6  2
3  4  6  0  1  4
4  2  4  2  0  2
5  4  1  6  4  0


### Optimal policy function by steps

In [6]:
def opf(it: int, cityJ: str, nodes: list, inner=False):
    minDist = []
    lowest = ""

    if it.__eq__(0):
        dist = G.get_edge_data(node[0][0], cityJ)["weight"]
    else:
        subDist = {}
        for city in nodes:
            subset = copy.deepcopy(nodes)
            subset.remove(city)
            dist1 = opf(it-1, city, subset, True)[0]
            dist2 = G.get_edge_data(city, cityJ)["weight"]
            subDist[city] = dist1 + dist2
            if not inner:
                minDist.append(f'{dist1} + {dist2}')
        lowest, dist = min(subDist.items(), key=lambda x: x[1])

    return dist, lowest, minDist


In [7]:
seed = df.index.to_list()[1:]

for i in seed:
    minDist, source, minSort = opf(0, str(i), [])
    print(f'f_0({i}, _) = {minDist}(1)')

for i in range(1, len(df) - 1):
    for j in seed:
        for k in list(it.combinations([item for item in seed if item not in j], i)):
            minDist, source, minSort = opf(i, str(j), list(k))
            print(f'f_{i}({j}, {list(k)}) = min[{minSort}] = {minDist}({source})')

minDist, source, minSort = opf(len(df) - 1, str(1), seed)
print(f'f_{len(df) - 1}(1, {seed}) = min[{minSort}] = {minDist}({source})')


f_0(2, _) = 2(1)
f_0(3, _) = 4(1)
f_0(4, _) = 4(1)
f_0(5, _) = 6(1)
f_1(2, ['3']) = min[['4 + 6']] = 10(3)
f_1(2, ['4']) = min[['4 + 4']] = 8(4)
f_1(2, ['5']) = min[['6 + 1']] = 7(5)
f_1(3, ['2']) = min[['2 + 4']] = 6(2)
f_1(3, ['4']) = min[['4 + 2']] = 6(4)
f_1(3, ['5']) = min[['6 + 6']] = 12(5)
f_1(4, ['2']) = min[['2 + 6']] = 8(2)
f_1(4, ['3']) = min[['4 + 1']] = 5(3)
f_1(4, ['5']) = min[['6 + 4']] = 10(5)
f_1(5, ['2']) = min[['2 + 2']] = 4(2)
f_1(5, ['3']) = min[['4 + 4']] = 8(3)
f_1(5, ['4']) = min[['4 + 2']] = 6(4)
f_2(2, ['3', '4']) = min[['6 + 6', '5 + 4']] = 9(4)
f_2(2, ['3', '5']) = min[['12 + 6', '8 + 1']] = 9(5)
f_2(2, ['4', '5']) = min[['10 + 4', '6 + 1']] = 7(5)
f_2(3, ['2', '4']) = min[['8 + 4', '8 + 2']] = 10(4)
f_2(3, ['2', '5']) = min[['7 + 4', '4 + 6']] = 10(5)
f_2(3, ['4', '5']) = min[['10 + 2', '6 + 6']] = 12(4)
f_2(4, ['2', '3']) = min[['10 + 6', '6 + 1']] = 7(3)
f_2(4, ['2', '5']) = min[['7 + 6', '4 + 4']] = 8(5)
f_2(4, ['3', '5']) = min[['12 + 1', '8 + 4']] = 12

### Shortest tour

In [8]:
path = nx.algorithms.approximation.threshold_accepting_tsp(G, "greedy", source="1")
path


['1', '3', '4', '5', '2', '1']

### Graph by edge list

In [9]:
for edge in G.edges(data=True):
    print(f'{edge[0]} {edge[1]} {edge[-1]["weight"]}')

1 2 2
1 3 4
1 4 4
1 5 6
2 1 4
2 3 4
2 4 6
2 5 2
3 1 4
3 2 6
3 4 1
3 5 4
4 1 2
4 2 4
4 3 2
4 5 2
5 1 4
5 2 1
5 3 6
5 4 4
